## Import Essential Libraries and Load in Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('E:\josh_data_science\Vassar Datafest 2018\Dataset_and_FeatureDictionary\indeed_cleaned.csv', encoding='cp1252')

In [3]:
# Zillow Median Monthly Rent Data by city from 2010 to January 2017
rent_df =\
pd.read_csv('E:\josh_data_science\Vassar Datafest 2018\Dataset_and_FeatureDictionary\Zillow Rent Price Data 2010-2017\price.csv')

In [4]:
# Remove unnecessary column
del df['Unnamed: 0']

In [5]:
df_us = df.copy()[df.copy().country=='US']

## Where are Best Cites to Work for recent grads in the U.S.?

Good places to work for recent graduates include the following conditions:
- Available job numbers for recent graduates (experienceRequired < 3) are plentiful
- Median Salary normalized by cost of rent is high
- Median avgOverallRating is high
- Has considerable median salary Increase if you have a college degree(higher education) than just with a high school degree

In [6]:
# Dropping all data with non-geographical info
df_us = df_us.copy()[~df_us.copy().city.isin(['Field', 'Home', 'Remote', 'UNKNOWN'])]

###### Available job posting numbers for recent graduates (experienceRequired < 3)

In [7]:
# Available job posting numbers for recent graduates (experienceRequired < 3) per city
df_us_job_num_count = df_us.copy()[df_us.copy().experienceRequired < 3].\
groupby(['stateProvince','city']).count().reset_index()[['stateProvince','city','date']]

df_us_job_num_count = df_us_job_num_count.rename(columns={'date':'avail_job_num'})

###### Median avgOverallRating

In [8]:
# Median of Average Overall Rating of companies per city
df_us_avgRating_median = \
df_us.copy()[df_us.copy().avgOverallRating!=0.0].groupby(['stateProvince','city']).avgOverallRating.median().reset_index()
df_us_avgRating_median = df_us_avgRating_median.rename(columns={'avgOverallRating':'avgOverallRating_median'})

###### Difference of Median Salary between job postings requiring college degree(higher education) and job postings that just require high school education

In [22]:
# Difference of Median Salary between Higher Education requiring jobs and high school education requiring jobs per city

df_us_median_salary_diff = pd.concat(
    [df_us.copy()[df_us.copy().educationRequirements.isin(['High School','Higher Education'])].\
             groupby(['stateProvince','city','educationRequirements']).estimatedSalary.median().reset_index(),
     df_us.copy()[df_us.copy().educationRequirements.isin(['High School','Higher Education'])].\
             groupby(['stateProvince','city','educationRequirements']).estimatedSalary.median().reset_index().\
    groupby(['stateProvince','city']).diff()], axis=1)

In [23]:
salary_med_diff = df_us_median_salary_diff.iloc[:,[-1]]
salary_med_diff = salary_med_diff.rename(columns={'estimatedSalary':'salary_med_diff'})

In [30]:
df_us_median_salary_diff =\
pd.concat([df_us_median_salary_diff.iloc[:,:2],salary_med_diff], axis=1).groupby(['stateProvince','city']).last().reset_index()

###### Median Salary normalized by cost of rent

In [12]:
rent_df_2017 = rent_df.copy().loc[:,['State','City','County','January 2017']]

In [13]:
rent_df_2017 = rent_df_2017.groupby(['State','City'])['January 2017'].mean().to_frame().reset_index()

In [14]:
df_us_rent = df_us.copy()[df_us.copy().city.isin(list(set(df_us.city.tolist()).intersection(set(rent_df_2017.City.tolist()))))]

In [15]:
rent_df_2017.columns = ['stateProvince','city','rent']

In [16]:
df_us_rent_salary_med = df_us_rent.groupby(['stateProvince','city']).estimatedSalary.median().to_frame().reset_index()
df_us_rent_salary_med = df_us_rent_salary_med.merge(rent_df_2017,on=['stateProvince','city'],how='left')

In [33]:
df_us_rent_salary_med['salary_normalized'] = df_us_rent_salary_med.estimatedSalary / df_us_rent_salary_med.rent

In [35]:
df_us_rent_salary_med_normalized = df_us_rent_salary_med.loc[:,['stateProvince','city','salary_normalized']]

###### Combining all of them into one dataframe for Tableau visualization

In [38]:
us_city_data = \
df_us_job_num_count.merge(df_us_avgRating_median, on=['stateProvince','city'],how='outer').\
merge(df_us_median_salary_diff,on=['stateProvince','city'],how='outer').\
merge(df_us_rent_salary_med_normalized,on=['stateProvince','city'],how='outer')

In [46]:
# Data that has all information filled in (not even single missing value for any of the columns)

us_city_data_non_null = us_city_data.dropna(how='any').reset_index()

In [48]:
## Saving File to local E-Drive for Tableau use
# us_city_data_non_null.to_csv('E:/josh_data_science/Vassar Datafest 2018/Dataset_and_FeatureDictionary/best_us_cities_to_work.csv')

###### Tableau Visualization

https://public.tableau.com/profile/seungjun.kim#!/vizhome/Indeed_comDataMapVisualization-WherearebestcitiestoworkinU_S_forrecentgrads/JobAvailabilityNormalizedMedianSalary?publish=yes

https://public.tableau.com/profile/seungjun.kim#!/vizhome/Indeed_comDataMapVisualization-WherearebestcitiestoworkinU_S_forrecentgrads2/ValueofCollegeDegreeOverallCompanyRatings?publish=yes

###### Top 10 Cities with highest availability of entry level jobs for recent graduates 

In [68]:
us_city_data_non_null.sort_values(['avail_job_num'], ascending=False).iloc[:10].loc[:,['stateProvince','city']]

,stateProvince,city
2711,NY,New York
1117,IL,Chicago
3557,TX,Houston
323,CA,Los Angeles
877,GA,Atlanta
3639,TX,San Antonio
3469,TX,Austin
3512,TX,Dallas
128,AZ,Phoenix
528,CO,Denver


###### Top 10 Cities with highest median avgOverallRating of companies

In [70]:
us_city_data_non_null.sort_values(['avgOverallRating_median'], ascending=False).iloc[:10].loc[:,['stateProvince','city']]

,stateProvince,city
2755,NY,West Hempstead
1138,IL,East Saint Louis
2595,NM,Hurley
1596,MA,Holbrook
3940,WA,Washougal
1882,MI,Jonesville
1524,LA,Jeanerette
3345,SC,Little River
3890,WA,La Center
1527,LA,La Place


###### Top 10 Cities where college degree is most valuable

In [71]:
us_city_data_non_null.sort_values(['salary_med_diff'], ascending=False).iloc[:10].loc[:,['stateProvince','city']]

,stateProvince,city
3810,VA,Pulaski
3143,PA,Flourtown
1700,MD,Charlotte Hall
3567,TX,Kerrville
265,CA,Grass Valley
2060,MN,Shoreview
3592,TX,Madisonville
2938,OH,Swanton
1244,IL,Quincy
3600,TX,Mexia


###### Top 10 Cities with highest median salary normalized by rent

In [72]:
us_city_data_non_null.sort_values(['salary_normalized'], ascending=False).iloc[:10].loc[:,['stateProvince','city']]

,stateProvince,city
3933,WA,Toppenish
1885,MI,Kincheloe
2076,MO,Berkeley
3187,PA,Marietta
1839,MI,Constantine
1445,KS,South Hutchinson
51,AL,Tallassee
1502,KY,Tompkinsville
3260,PA,Upland
2321,NC,Spindale


## Which cities get the most interest from local applicants?

In [74]:
df_us_local_prop = df_us.copy()
df_us_local_prop['localClicks/clicks'] = df_us_local_prop.localClicks / df_us_local_prop.clicks

In [80]:
df_us_local_prop_sum_mean = df_us_local_prop.groupby(['stateProvince','city'])['localClicks/clicks'].agg({'sum','mean'}).reset_index().\
sort_values('mean',ascending=False)

In [86]:
df_us_local_prop_sum_mean = df_us_local_prop_sum_mean.copy()[df_us_local_prop_sum_mean.copy()['mean']!=1.0]

In [89]:
df_us_local_prop_sum_mean = df_us_local_prop_sum_mean.copy()[df_us_local_prop_sum_mean.copy()['sum']!=
                                                            df_us_local_prop_sum_mean.copy()['mean']]

In [92]:
df_us_local_prop_sum_mean.sort_values('mean',ascending=False).iloc[:20][['stateProvince','city']]

,stateProvince,city
8041,MO,Theodosia
3164,GA,Sylvania
3828,IL,Big Rock
7386,MN,Hokah
4089,IL,Hume
1336,CA,Nuevo
3342,IA,Baldwin
10935,NY,Washingtonville
7713,MO,Chilhowee
16098,WI,Collins
